# Agentic Retrieval Quickstart for Azure AI Search

### 1. Load Connections

In [13]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

load_dotenv(override=True) # take environment variables from .env.

# The following variables from your .env file are used in this notebook
answer_model = os.getenv("ANSWER_MODEL", "gpt-4o")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "earth_at_night")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4o")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4o")
azure_openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION", "2025-03-01-preview")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "earth-search-agent")
api_version = "2025-05-01-Preview"

### 2. Create search agent

In [14]:
import requests

create_agent_request = {
    "name": agent_name,
    "targetIndexes": [ { "indexName": index_name } ],
    "models": [
          {
            "kind": "azureOpenAI",
            "azureOpenAIParameters": {
                "resourceUri": azure_openai_endpoint,
                "apiKey": None,
                "deploymentId": azure_openai_gpt_model,
                "modelName": azure_openai_gpt_model
            }
        }
    ]
}

response = requests.put(
    url=f"{endpoint}/agents/{agent_name}?api-version={api_version}",
    headers={ "Authorization": f"Bearer {token_provider()}" },
    json=create_agent_request
)
response.raise_for_status()


### 3. Setup messages

In [15]:
instructions = """
An Q&A agent that can answer questions about the Earth at night.
Sources have a JSON format with a ref_id that must be cited in the answer.
If you do not have the answer, respond with "I don't know".
"""

messages = [
    {
        "role": "system",
        "content": instructions
    }
]

### 4. Use Agentic Retrieval to fetch results

In [16]:
def query_agent(messages: list[dict[str, any]]):
    retrieval_request = {
        "messages" : [ { "role": msg["role"], "content": [ { "text": msg["content"] , "type": "text" } ] } for msg in messages ],
        "targetIndexParams" :  [
            { 
                "indexName" : index_name,
                "rerankerThreshold": 2.5
            } 
        ]
    }
    response = requests.post(
        url=f"{endpoint}/agents/{agent_name}/retrieve?api-version={api_version}",
        headers={ "Authorization": f"Bearer {token_provider()}" },
        json=retrieval_request
    )
    response.raise_for_status()
    result = response.json()
    return result

In [17]:
messages.append({
    "role": "user",
    "content": """
    Why do suburban belts display larger December brightening than urban cores even though absolute light levels are higher downtown?
    Why is the Phoenix nighttime street grid is so sharply visible from space, whereas large stretches of the interstate between midwestern cities remain comparatively dim?
    """
})

retrieval_result = query_agent(messages)
messages.append({
    "role": "assistant",
    "content": retrieval_result["response"][0]["content"][0]["text"]
})

### 4.1. Review retrieval activity and results

In [18]:
import json

print("Retrieval activity")
print(json.dumps(retrieval_result["activity"], indent=2))
print("Retrieval results")
print(json.dumps(retrieval_result["references"], indent=2))

Retrieval activity
[
  {
    "type": "ModelQueryPlanning",
    "id": 0,
    "inputTokens": 1407,
    "outputTokens": 526
  },
  {
    "type": "AzureSearchQuery",
    "id": 1,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "suburban belts December brightening compared to urban cores",
      "filter": null
    },
    "queryTime": "2025-04-30T04:36:33.405Z",
    "elapsedMs": 581
  },
  {
    "type": "AzureSearchQuery",
    "id": 2,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "Phoenix nighttime street grid visibility from space",
      "filter": null
    },
    "queryTime": "2025-04-30T04:36:33.687Z",
    "count": 2,
    "elapsedMs": 281
  },
  {
    "type": "AzureSearchQuery",
    "id": 3,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "interstate visibility between midwestern cities at night",
      "filter": null
    },
    "queryTime": "2025-04-30T04:36:33.925Z",
    "count": 2,
    "elapsedMs": 238
  }
]
Retrieval resul

### 5. Create Azure OpenAI Client

In [19]:
from openai import AzureOpenAI
from azure.identity import get_bearer_token_provider

azure_openai_token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
client = AzureOpenAI(
    azure_endpoint=azure_openai_endpoint,
    azure_ad_token_provider=azure_openai_token_provider,
    api_version=azure_openai_api_version
)

### 5.1 Use Responses API to generate an answer

In [20]:
import textwrap

response = client.responses.create(
    model=answer_model,
    input=messages
)

wrapped = textwrap.fill(response.output_text, width=100)
print(wrapped)

Suburban belts display larger December brightening than urban cores despite higher absolute light
levels downtown due to several factors. Suburban areas often experience increased holiday lighting
during December, enhancing their visibility more than urban cores. Urban areas already have higher
baseline illumination and infrastructure lighting, so the additional holiday lights make a more
noticeable relative increase in suburbs than in urban centers [ref_id: 0].  The Phoenix nighttime
street grid is sharply visible from space due to the regular grid layout and dense street lighting
characteristic of western U.S. cities. The lighting patterns follow major transport corridors and
intersections, making them clearly identifiable. In contrast, large stretches of the interstate
between midwestern cities remain dim because these areas are less densely populated and lack the
same level of street and infrastructure lighting seen in urban settings like Phoenix [ref_id: 1, 3].


### 5.2 Use Chat Completions API to generate an answer

In [21]:
response = client.chat.completions.create(
    model=answer_model,
    messages=messages
)

wrapped = textwrap.fill(response.choices[0].message.content, width=100)
print(wrapped)

Suburban belts often display larger December brightening than urban cores due to the difference in
lighting infrastructure and residential patterns. Suburban areas tend to have more individual houses
with outdoor lighting, decorations, and streetlights, which are especially prominent during the
holiday season. In contrast, urban cores, though having higher absolute light levels overall,
contain more commercial and industrial areas where additional holiday lighting might be less common
and noticeable. Therefore, during December, the increase in lighting per residence in suburban areas
contributes to greater perceived brightening compared to urban cores [ref_id:3].  In contrast, the
Phoenix nighttime street grid is sharply visible from space due to its distinctive layout and
extensive street lighting that follows the urban grid pattern typical of western U.S. cities. The
streets of Phoenix are lined with bright lights, making the grid structure prominent from a
satellite view. Interstate

### 6. Continue the conversation

In [22]:
messages.append({
    "role": "user",
    "content": "How do I find lava at night?"
})

retrieval_result = query_agent(messages)
messages.append({
    "role": "assistant",
    "content": retrieval_result["response"][0]["content"][0]["text"]
})

### 6.1. Review activity and results

In [23]:
print("Retrieval activity")
print(json.dumps(retrieval_result["activity"], indent=2))
print("Retrieval results")
print(json.dumps(retrieval_result["references"], indent=2))

Retrieval activity
[
  {
    "type": "ModelQueryPlanning",
    "id": 0,
    "inputTokens": 3540,
    "outputTokens": 104
  },
  {
    "type": "AzureSearchQuery",
    "id": 1,
    "targetIndex": "earth_at_night",
    "query": {
      "search": "finding lava at night",
      "filter": null
    },
    "queryTime": "2025-04-30T04:36:43.875Z",
    "count": 6,
    "elapsedMs": 1546
  }
]
Retrieval results
[
  {
    "type": "AzureSearchDoc",
    "id": "0",
    "activitySource": 1,
    "docKey": "earth_at_night_508_page_64_verbalized",
    "sourceData": null
  },
  {
    "type": "AzureSearchDoc",
    "id": "1",
    "activitySource": 1,
    "docKey": "earth_at_night_508_page_65_verbalized",
    "sourceData": null
  },
  {
    "type": "AzureSearchDoc",
    "id": "2",
    "activitySource": 1,
    "docKey": "earth_at_night_508_page_44_verbalized",
    "sourceData": null
  },
  {
    "type": "AzureSearchDoc",
    "id": "3",
    "activitySource": 1,
    "docKey": "earth_at_night_508_page_66_verbaliz

### 6.2. Generate answer

In [24]:
response = client.responses.create(
    model=answer_model,
    input=messages
)

wrapped = textwrap.fill(response.output_text, width=100)
print(wrapped)

To find lava at night, you can use thermal imaging and satellite data. Satellites equipped with
thermal infrared sensors, like the Landsat 8 or VIIRS, can detect the heat emitted by lava flows,
making them visible even in the absence of daylight. These instruments capture the infrared
signatures of the hot lava, which appear bright against cooler surroundings, allowing tracking and
monitoring for scientific research and safety purposes. This capability is particularly useful
during volcanic eruptions, such as those observed on Mount Etna or Kilauea, where thermal data helps
visualize active lava flows. [ref_id:5][ref_id:4].
